In [1]:
import cv2
import numpy as np
import time

In [2]:
apple = cv2.imread('apple.png')
orange = cv2.imread('orange.png')
# cv2.imshow('apple', apple)
# cv2.imshow('orange', orange)
# key = cv2.waitKey()
# if key == 27:
#     cv2.destroyAllWindows()

In [3]:
yjl = cv2.imread('official_photo.jpg')
yy = cv2.imread('yy_official.jpg')
print(yjl.shape, yy.shape)
# cv2.imshow('apple', yjl)
# cv2.imshow('orange', yy)
# key = cv2.waitKey()
# if key == 27:
#     cv2.destroyAllWindows()

(567, 390, 3) (605, 468, 3)


In [4]:
# resize the longer side to a given length and zero pad to square
def resize2square(img, length=512):
    outcome = img.copy()
    rows, cols, chnls = img.shape
    longer_length = max(rows, cols)
    new_rows = int(rows * length / longer_length)
    new_cols = int(cols * length / longer_length)
    outcome = cv2.resize(outcome, (new_cols, new_rows))
    diff_rows = length - new_rows
    diff_cols = length - new_cols
    top, bottom = diff_rows // 2 , diff_rows-(diff_rows // 2)
    left, right = diff_cols//2, diff_cols-(diff_cols//2)
    # print(top, bottom, left, right)
    outcome = cv2.copyMakeBorder(outcome, top, bottom, left, right, cv2.BORDER_CONSTANT)
    return outcome
    

In [5]:
def pyramid_blending(apple, orange, n): # how many layers of pyramid we want? n layers
    apple_copy = resize2square(apple.copy())
    assert 2**(n-1) <= apple_copy.shape[0] # to rule out unexpected situation, make sure the last img is >= 1 pixel
    orange_copy = resize2square(orange.copy())
    gauss_pyrmd = []

    for i in range(n):
        gauss_pyrmd.append((apple_copy, orange_copy))
        print(apple_copy.shape)
    #     cv2.imshow('apple'+str(i), apple_copy)
    #     cv2.imshow('orange'+str(i), orange_copy)
        apple_copy = cv2.pyrDown(apple_copy)
        orange_copy = cv2.pyrDown(orange_copy)
    apple_laplac, orange_laplac = [], []
    for i in range(n-1):
        apple_up = cv2.pyrUp(gauss_pyrmd[i+1][0])
        apple_up = cv2.resize(apple_up, (gauss_pyrmd[i][0].shape[1],gauss_pyrmd[i][0].shape[0]), interpolation = cv2.INTER_AREA)
        apple_laplac.append(gauss_pyrmd[i][0] - apple_up)
        orange_up = cv2.pyrUp(gauss_pyrmd[i+1][1])
        orange_up = cv2.resize(orange_up,(gauss_pyrmd[i][1].shape[1],gauss_pyrmd[i][1].shape[0]), interpolation = cv2.INTER_AREA)
        orange_laplac.append(gauss_pyrmd[i][1] - orange_up)

    # for i, j in zip(apple_laplac, orange_laplac):
    #     cv2.imshow(str(i), i)
    #     cv2.imshow(str(j), j)
    mid = int(gauss_pyrmd[-1][0].shape[1]/2)
    outcome = np.hstack((gauss_pyrmd[-1][0][:,:mid], gauss_pyrmd[-1][1][:,mid:]))

    for i in range(n-2, -1, -1):
        outcome = cv2.pyrUp(outcome)
        outcome = cv2.resize(outcome, (gauss_pyrmd[i][0].shape[1],gauss_pyrmd[i][0].shape[0]), interpolation = cv2.INTER_AREA)
        mid = int(outcome.shape[1]/2)
        outcome = outcome + np.hstack((apple_laplac[i][:,:mid], orange_laplac[i][:,mid:]))
    return outcome

In [6]:
outcome = pyramid_blending(yjl, yy, 6)
cv2.imshow('orange', outcome)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

(512, 512, 3)
(256, 256, 3)
(128, 128, 3)
(64, 64, 3)
(32, 32, 3)
(16, 16, 3)
